In [161]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')
os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')

In [117]:
pd.options.display.max_columns = None
#Home
#telco=pd.read_csv('Telco_Customer_Attr.csv')
#Office
telco=pd.read_csv('Teleco_Cust_Attr.csv')
telco.head()
#print(telco.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Fixing the missing values using the Stratified mean value imputation

In [118]:
Mean_No_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['No']
Mean_Yes_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['Yes']

In [119]:
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='No'),['TotalCharges']]=Mean_No_TotalCharges
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='Yes'),['TotalCharges']]=Mean_Yes_TotalCharges

Dropping the columns which dont explain much variance

In [120]:
telco.drop(['customerID','MonthlyCharges','TotalCharges','tenure'],axis=1,inplace=True)

Data type conversion

In [121]:
telco['SeniorCitizen']=telco['SeniorCitizen'].astype('object')

In [122]:
telco_backup=telco

In [123]:
y=telco['Churn']
telco=telco.drop('Churn',axis=1)

In [124]:
## Train Test Split

In [125]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import pandas as pd
encoding=defaultdict(LabelEncoder)
columns_nominal=['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']
for col in columns_nominal:
    #encoded_df=pd.DataFrame()
    encoding[col].fit(telco[col])
    telco[col]=encoding[col].transform(telco[col])

In [126]:
telco.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,0,0,1,0,0,1,0,0,2,0,0,0,0,0,1,2
1,1,0,0,0,1,0,0,2,0,2,0,0,0,1,0,3
2,1,0,0,0,1,0,0,2,2,0,0,0,0,0,1,3
3,1,0,0,0,0,1,0,2,0,2,2,0,0,1,0,0
4,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,2


In [127]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(telco,y,test_size=0.30,random_state=42)

In [128]:
### Applying the Navie Bayes Algorithm on the dataset

In [135]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
BNB=BernoulliNB()
parameters={'alpha':[1,2,3,6,12,15]}
GSCV_BNB=GridSearchCV(BNB,parameters)
GSCV_BNB.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [1, 2, 3, 6, 12, 15]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [136]:
GSCV_BNB.

{'alpha': 1}

In [142]:
from sklearn.naive_bayes import BernoulliNB
BNB=BernoulliNB(alpha=1)
BNB.fit(X_train,y_train)
Train_Predicted_BNB=BNB.predict(X_train)
Test_Predicted_BNB=BNB.predict(X_test)

In [145]:
from sklearn.metrics import confusion_matrix
CF_train=confusion_matrix(y_train,Train_Predicted_BNB)
CF_test=confusion_matrix(y_test,Test_Predicted_BNB)
print('Confusion Metrics:\n',CF_train)
print('Confusion Metrics: \n', CF_test)

import numpy as np
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score,recall_score
train_accuracy=accuracy_score(y_train,Train_Predicted_BNB)
test_accuracy=accuracy_score(y_test,Test_Predicted_BNB)
print('Train accuracy:',train_accuracy)
print('Test accuracy:',test_accuracy)

train_precision=precision_score(y_train,Train_Predicted_BNB,average='macro')
test_precision=precision_score(y_test,Test_Predicted_BNB,average='macro')
print('Train precision:',train_precision)
print('Test precision:',test_precision)

train_recall=recall_score(y_train,Train_Predicted_BNB,average='macro')
test_recall=recall_score(y_test,Test_Predicted_BNB,average='macro')
print('Train recall:',train_recall)
print('Test recall:',test_recall)

Confusion Metrics:
 [[2789  846]
 [ 388  907]]
Confusion Metrics: 
 [[1167  372]
 [ 162  412]]
Train accuracy: 0.7496957403651116
Test accuracy: 0.747278750591576
Train precision: 0.6976354757463306
Test precision: 0.701807020776708
Train recall: 0.7338244119537105
Test recall: 0.7380273176165346


In [146]:
BNB.classes_

array(['No', 'Yes'], dtype='<U3')

In [147]:
BNB.feature_count_

array([[1894.,  462., 1923., 1262., 3281., 1882., 2264., 2203., 2342.,
        2289., 2199., 2347., 2345., 2082., 1926., 2735.],
       [ 651.,  328.,  469.,  229., 1182.,  701.,  972.,  288.,  448.,
         461.,  306.,  636.,  657.,  158.,  961., 1109.]])

In [148]:
BNB.feature_log_prob_

array([[-0.65194033, -2.06118739, -0.63675281, -1.05766932, -0.10270617,
        -0.65829292, -0.47358441, -0.50088527, -0.43972701, -0.46260735,
        -0.50270181, -0.43759526, -0.43844742, -0.55735   , -0.63519478,
        -0.28466217],
       [-0.68776462, -1.37175143, -1.01507649, -1.72972988, -0.09200032,
        -0.61387578, -0.2874251 , -1.5013825 , -1.0607863 , -1.03224429,
        -1.44096144, -0.71103953, -0.67860425, -2.09890498, -0.29879473,
        -0.15569389]])

In [149]:
BNB.class_log_prior_

array([-0.30472988, -1.33682829])

In [150]:
BNB.get_params

<bound method BaseEstimator.get_params of BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)>

In [151]:
BNB.score

<bound method ClassifierMixin.score of BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)>

In [152]:
BNB.coef_

array([[-0.68776462, -1.37175143, -1.01507649, -1.72972988, -0.09200032,
        -0.61387578, -0.2874251 , -1.5013825 , -1.0607863 , -1.03224429,
        -1.44096144, -0.71103953, -0.67860425, -2.09890498, -0.29879473,
        -0.15569389]])

In [155]:
BNB.predict_proba

<bound method BaseNB.predict_proba of BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)>

In [156]:
BNB.predict_log_proba

<bound method BaseNB.predict_log_proba of BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)>

In [160]:
Test_Predict_Prob=BNB.predict_proba(X_test)
print(Test_Predict_Prob)
Test_Predict_log_Prob=BNB.predict_log_proba(X_test)
print(Test_Predict_log_Prob)
print(Test_Predicted_BNB)

[[6.89321384e-02 9.31067862e-01]
 [9.72805324e-01 2.71946762e-02]
 [9.99551477e-01 4.48522595e-04]
 ...
 [9.98075514e-01 1.92448582e-03]
 [1.02734831e-01 8.97265169e-01]
 [4.87318524e-01 5.12681476e-01]]
[[-2.67463276e+00 -7.14231133e-02]
 [-2.75712951e-02 -3.60473405e+00]
 [-4.48623212e-04 -7.70955150e+00]
 ...
 [-1.92634002e-03 -6.25309646e+00]
 [-2.27560407e+00 -1.08403843e-01]
 [-7.18837315e-01 -6.68100532e-01]]
['Yes' 'No' 'No' ... 'No' 'Yes' 'Yes']


In [ ]:
## Model Important Features

In [167]:
a=BNB.feature_log_prob_[0,:].argsort()
print(a)
telco.columns
np.take(telco.columns,a[:5])

[ 1  3  5  0  2 14 13 10  7  6  9  8 12 11 15  4]


Index(['SeniorCitizen', 'Dependents', 'MultipleLines', 'gender', 'Partner'], dtype='object')

In [ ]:
BNB.